# ĐỒ ÁN THỰC HÀNH - NMKHDL

# Khám phá dữ liệu

## I. Tổng quan về bộ dữ liệu

### 1. Đọc dữ liệu và tính số dòng và cột

Đầu tiên ta import các thư viện cần thiết.

In [2]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# .. import more libraries

import seaborn as sns
sns.set_theme()

# remove warnings
import warnings
warnings.simplefilter('ignore')
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import iplot
import plotly.io as pio
from scipy.spatial.distance import cdist

sns.set_style("white")
sns.set(rc={"axes.facecolor":"#F2EAC5","figure.facecolor":"#F2EAC5"})
sns.set_context("poster",font_scale = .7)

palette = ["#11264e","#00507A","#026e90","#008b99","#6faea4","#fcdcb0","#FEE08B","#faa96e","#f36b3b","#ef3f28","#CC0028"]
palette_cmap=["#CC0028","#ef3f28","#f36b3b","#faa96e","#FEE08B","#fcdcb0","#6faea4","#008b99","#026e90","#00507A","#11264e"]


Tiếp đến đọc file `"chotot.csv"` vào dataframe `house_df` và in ra 5 dòng đầu tiên của dataframe.

In [3]:
house_df = pd.read_csv('chotot.csv')
house_df.head()

,DiaChi,GiaBan,DuAn,DienTich,TinhTrangBDS,Gia/m2,PhongNgu,PhongVeSinh,SoTang,TinhTrangGiayTo,TinhTrangNoiThat,HuongBanCong,HuongCua,DacDiem
0,"Đường Huy Cận, Phường Phước Long B (Quận 9 cũ)...","2,35 tỷ",The Art,66 m²,Chưa bàn giao,"35,61 triệu/m²",2 phòng,2 phòng,8.0,Đã có sổ,Nội thất cao cấp,Tây Bắc,Đông Bắc,NaN
1,"số 88, số 88 Đường N1, Phường Sơn Kỳ, Quận Tân...","3,42 tỷ",Celadon City,71 m²,Đã bàn giao,"48,17 triệu/m²",2 phòng,2 phòng,NaN,Đang chờ sổ,Nội thất cao cấp,NaN,NaN,NaN
2,"Võ Văn Kiệt, Phường An Lạc, Quận Bình Tân, Tp ...","2,4 tỷ",Akari City,56 m²,Đã bàn giao,"42,86 triệu/m²",2 phòng,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Bát Nàn, Phường Thạnh Mỹ Lợi (Quận 2 cũ), Thàn...","7,5 tỷ",One Verandah Mapletree,107 m²,Đã bàn giao,"70,09 triệu/m²",3 phòng,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Võ Văn Kiệt, Phường An Lạc, Quận Bình Tân, Tp ...","3,1 tỷ",Akari City,75 m²,Chưa bàn giao,"41,33 triệu/m²",2 phòng,2 phòng,NaN,NaN,NaN,NaN,NaN,NaN


Tính số dòng và số cột và lưu vào 2 biến `num_rows` và `num_cols`.

In [4]:
num_rows, num_cols = house_df.shape
print(f'Number of rows: {num_rows}\nNumber of columns: {num_cols}')

Number of rows: 1135
Number of columns: 14


### 2. Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Như đã được đề cập ở trên, dữ liệu được thu thập bằng cách crawl dữ liệu thô trên trang web 'https://www.nhatot.com/mua-ban-can-ho-chung-cu-tp-ho-chi-minh'. Mỗi dòng trong tập dữ liệu này tương ứng với một bản ghi các thông tin về căn hộ chung cư ở TP.HCM được đăng bán trên trang web 'chotot'. Bao gồm các thuộc tính như: địa chỉ, giá bán, dự án thầu, tình trạng bàn giao, số phòng ngủ, số phòng vệ sinh, giá/m2, căn hộ ở tầng mấy của chung cư, tình trang giấy tờ, tình trạng nội thất, hướng ban công, hướng cửa chính, là căn góc hay trung tâm. Do vậy, ta có thể thấy rằng không có dòng nào bị "lạc loài".

### 3. Dữ liệu có các dòng bị lặp không?

Ta kiểm tra xem có dòng nào bị lặp không bằng cách sử dụng các phương `duplicated()` và `any()` trên dataframe `house_df` và lưu kết quả vào biến `have_duplicated_rows`. Biến này sẽ có giá trị True nếu dữ liệu có các dòng bị lặp và có giá trị False nếu ngược.

In [5]:
have_duplicate_rows = house_df.duplicated().any()
have_duplicate_rows

False

Do vậy, ta có thể thấy rằng dữ liệu không có các dòng bị lặp. Dễ thấy do thông tin các căn hộ được đăng bán là khác nhau và duy nhất cho một người chủ đăng bán.

### 4. Tỉ lệ giá trị thiếu và thống kê mô tả của từng cột

Ta cũng cần xem xét đến các giá trị thiếu trong dữ liệu. Đầu tiên, ta tính tỉ lệ giá trị thiếu của từng cột bằng cách sử dụng phương thức `isnull()` trên dataframe `house_df` và tính tổng số giá trị thiếu của từng cột bằng phương thức `sum()`. Cuối cùng ta chia số dòng và lưu kết quả vào `missing_ratio`.

In [6]:
missing_ratio = house_df.isnull().sum()
missing_ratio = missing_ratio / num_rows
missing_ratio

DiaChi              0.000000
GiaBan              0.000000
DuAn                0.000000
DienTich            0.000000
TinhTrangBDS        0.000881
Gia/m2              0.000000
PhongNgu            0.000000
PhongVeSinh         0.166520
SoTang              0.751542
TinhTrangGiayTo     0.496916
TinhTrangNoiThat    0.536564
HuongBanCong        0.748899
HuongCua            0.796476
DacDiem             0.824670
dtype: float64

- Có thể thấy, dữ liệu có rất nhiều thuộc tính bị thiếu và tỉ lệ thiếu khá lớn. Do vậy, việc tiền xử lý cho bộ dữ liệu này là vô cùng cần thiết.
- Sở dĩ việc có nhiều giá trị thiếu do mỗi người đăng thông tin lên web có các thông tin cho ngôi nhà khác nhau, và trang web không có một form bắt buộc cho các bài được đăng nên có những thông tin ngôi nhà này có cho biết nhưng ngôi nhà khác lại không cho biết.

Ta cũng cần tính các giá trị thống kê mô tả của các cột numeric bằng phương thức `describe()` trên dataframe `house_df`.

In [7]:
house_df.describe()

,SoTang
count,282.000000
mean,13.386525
std,20.639670
min,1.000000
25%,7.000000
50%,10.000000
75%,16.000000
max,321.000000


- Do hiện tại bộ dữ liệu chỉ có một cột SoTang là có kiểu dữ liệu numerical nên dùng phương thức describe() sẽ chỉ thống kê cho mỗi cột SoTang.
- Để xem thống kê cho các cột không phải dạng numerical ta phải gọi cụ thể cột đó.

In [8]:
cate_col_df=house_df.select_dtypes(exclude=['float64','float32','int64','int32'])
cate_col_df.describe()

,DiaChi,GiaBan,DuAn,DienTich,TinhTrangBDS,Gia/m2,PhongNgu,PhongVeSinh,TinhTrangGiayTo,TinhTrangNoiThat,HuongBanCong,HuongCua,DacDiem
count,1135,1135,1135,1135,1134,1135,1135,946,571,526,285,231,199
unique,452,330,311,173,2,866,5,5,3,4,8,8,1
top,"Số 4, Đào Trí, Phường Phú Thuận, Quận 7, Tp Hồ...","2,1 tỷ",Saigon Riverside Complex Q7,70 m²,Đã bàn giao,50 triệu/m²,2 phòng,2 phòng,Đã có sổ,Nội thất đầy đủ,Đông Nam,Tây Bắc,Căn góc
freq,39,40,44,43,929,16,785,687,370,210,77,40,199


Ví dụ với cột GiaBan:
- Có 1137 dòng.
- Có 332 giá trị khác nhau.
- Giá trị xuất hiện nhiều nhất là 2,1 tỷ với tần suất xuất hiện là 40 lần.

### 5. Kiểu dữ liệu của mỗi cột

Ta sử dụng phương thức `dtypes` trên dataframe `house_df` để xem kiểu dữ liệu của mỗi cột. Kết quả được lưu vào series `col_dtypes`; series này có index là tên các cột và giá trị là kiểu dữ liệu của các cột tương ứng.

In [9]:
col_dtype = house_df.dtypes
col_dtype

DiaChi               object
GiaBan               object
DuAn                 object
DienTich             object
TinhTrangBDS         object
Gia/m2               object
PhongNgu             object
PhongVeSinh          object
SoTang              float64
TinhTrangGiayTo      object
TinhTrangNoiThat     object
HuongBanCong         object
HuongCua             object
DacDiem              object
dtype: object

#### Nhận xét
- Về mặt kiểu dữ liệu, các thuộc tính của tập dữ liệu có nhiều cột đang ở định dạng chưa phù hợp, nên cần phải tiền xử lý giai đoạn này.

### 6. Thay đổi kiểu dữ liệu phù hợp cho các cột.

- Các cột: **GiaBan, DienTich, Gia/m2, PhongNgu, PhongVeSinh** nên được đưa về kiểu dữ liệu numerical để tiếp tục xử lý.
- Về mặt thực tế những giá trị này là dạng số thực, và là những thuộc tính quan trọng liên quan đến giá căn hộ, do đó để thuận tiện cho việc áp dụng mô hình học máy thì nên đưa các cột này về dạng số.

In [10]:
#xử lý cho 4 cột 'GiaBan', 'DienTich', 'PhongNgu', 'PhongVeSinh'.
vals=house_df[['GiaBan', 'DienTich', 'PhongNgu', 'PhongVeSinh']].values
list_vals=vals.tolist()
list_num_vals=[[str(float(str(l).split()[0].replace(',','.'))/(10**3)) if (len(str(l).split())>1) and ('triệu' in str(l).split()[1])
                else str(l).split()[0].replace(',','.') for l in lv] for lv in list_vals]
house_df[['GiaBan', 'DienTich', 'PhongNgu', 'PhongVeSinh']]=list_num_vals
#xử lý cho cột 'Gia/m2'.
list_val=house_df['Gia/m2'].values.tolist()
list_num_val=[str(float(str(l).split()[0].replace(',','.'))*(10**3)) if (len(str(l).split())>1) and ('tỷ' in str(l).split()[1])
                else str(l).split()[0].replace(',','.') for l in list_val]
house_df['Gia/m2']=list_num_val
house_df.head(5)

,DiaChi,GiaBan,DuAn,DienTich,TinhTrangBDS,Gia/m2,PhongNgu,PhongVeSinh,SoTang,TinhTrangGiayTo,TinhTrangNoiThat,HuongBanCong,HuongCua,DacDiem
0,"Đường Huy Cận, Phường Phước Long B (Quận 9 cũ)...",2.35,The Art,66,Chưa bàn giao,35.61,2,2,8.0,Đã có sổ,Nội thất cao cấp,Tây Bắc,Đông Bắc,NaN
1,"số 88, số 88 Đường N1, Phường Sơn Kỳ, Quận Tân...",3.42,Celadon City,71,Đã bàn giao,48.17,2,2,NaN,Đang chờ sổ,Nội thất cao cấp,NaN,NaN,NaN
2,"Võ Văn Kiệt, Phường An Lạc, Quận Bình Tân, Tp ...",2.4,Akari City,56,Đã bàn giao,42.86,2,nan,NaN,NaN,NaN,NaN,NaN,NaN
3,"Bát Nàn, Phường Thạnh Mỹ Lợi (Quận 2 cũ), Thàn...",7.5,One Verandah Mapletree,107,Đã bàn giao,70.09,3,nan,NaN,NaN,NaN,NaN,NaN,NaN
4,"Võ Văn Kiệt, Phường An Lạc, Quận Bình Tân, Tp ...",3.1,Akari City,75,Chưa bàn giao,41.33,2,2,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
house_df[['GiaBan', 'DienTich', 'Gia/m2', 'PhongNgu', 'PhongVeSinh']]=house_df[['GiaBan', 'DienTich', 'Gia/m2', 'PhongNgu', 'PhongVeSinh']].astype('float64')
house_df.dtypes

DiaChi               object
GiaBan              float64
DuAn                 object
DienTich            float64
TinhTrangBDS         object
Gia/m2              float64
PhongNgu            float64
PhongVeSinh         float64
SoTang              float64
TinhTrangGiayTo      object
TinhTrangNoiThat     object
HuongBanCong         object
HuongCua             object
DacDiem              object
dtype: object

Ngoài ra cột địa chỉ chứa các giá trị quá riêng biệt, không có ý nghĩa cho việc trực quan hay phân tích. Do đó lấy địa chỉ một cách tổng quát là quận/huyện/thành phố trực thuộc và thêm vào dataframe cột 'Quan'.

In [24]:
df = house_df.copy()
addr=[]
for i in range(0,len(df['DiaChi'])):
    a= df.DiaChi[i].split(',')
    temp=[]
    for j in a:
        if 'Huyện' in j:
            temp.append(j)
        if 'Quận' in j:
            temp.append(j)
        if 'Thủ Đức' in j:
            temp.append(j)
    addr=addr+[temp[-1]]
house_df['Quan']=addr
house_df.head(5)

,DiaChi,GiaBan,DuAn,DienTich,TinhTrangBDS,Gia/m2,PhongNgu,PhongVeSinh,SoTang,TinhTrangGiayTo,TinhTrangNoiThat,HuongBanCong,HuongCua,DacDiem,Quan
0,"Đường Huy Cận, Phường Phước Long B (Quận 9 cũ)...",2.35,The Art,66.0,Chưa bàn giao,35.61,2.0,2.0,8.0,Đã có sổ,Nội thất cao cấp,Tây Bắc,Đông Bắc,NaN,Thành phố Thủ Đức
1,"số 88, số 88 Đường N1, Phường Sơn Kỳ, Quận Tân...",3.42,Celadon City,71.0,Đã bàn giao,48.17,2.0,2.0,NaN,Đang chờ sổ,Nội thất cao cấp,NaN,NaN,NaN,Quận Tân Phú
2,"Võ Văn Kiệt, Phường An Lạc, Quận Bình Tân, Tp ...",2.40,Akari City,56.0,Đã bàn giao,42.86,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quận Bình Tân
3,"Bát Nàn, Phường Thạnh Mỹ Lợi (Quận 2 cũ), Thàn...",7.50,One Verandah Mapletree,107.0,Đã bàn giao,70.09,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Thành phố Thủ Đức
4,"Võ Văn Kiệt, Phường An Lạc, Quận Bình Tân, Tp ...",3.10,Akari City,75.0,Chưa bàn giao,41.33,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,Quận Bình Tân


### 7. Xem xét sự phân bố giá trị của các cột dữ liệu dạng số.

**Trước khi xem xét sự phân bố, xem xét qua yếu tố sau đây:**
- Đối với cột GiaBan: Đa số giá trị được tính theo đơn vị là tỷ,nhưng sẽ có một số ít giá trị có đơn vị là triệu. Ví dụ giá trị lớn nhất thực tế là 130 tỷ nhưng vô tình giá trị 900 triệu nếu không được xử lý thì khi chuyển sang dạng số sẽ là giá trị lớn nhất. Từ việc này làm thay đổi min, max của cột dữ liệu do đó ta sẽ đổi tất cả giá trị theo đơn vị là triệu về tỷ.
- Tương tự dối với cột Gia/m2: Đa số giá trị được tính theo đơn vị là triệu,nhưng sẽ có một số ít giá trị có đơn vị là tỷ. Ví dụ giá trị nhỏ nhất thực tế là 1 triệu nhưng vô tình giá trị 0.8 tỷ nếu không được xử lý thì khi chuyển sang dạng số sẽ là giá trị nhỏ nhất. Từ việc này làm thay đổi min, max của cột dữ liệu do đó ta sẽ đổi tất cả giá trị theo đơn vị là triệu về triệu.

Ta thấy sau các bước tiền xử lý ở trên có 6 cột dữ liệu kiểu số là: GiaBan, DienTich, Gia/m2, PhongNgu, PhongVeSinh, SoTang.

Thực hiện thống kê trên 6 cột này và lưu vào một dataframe với các dòng là đại diện cho các giá trị như sau:
- Tỉ lệ % (từ 0 đến 100) các giá trị thiếu (missing_ratio).
- Giá trị min (min).
- Giá trị lower quartile (phân vị 25) (lower_quartile).
- Giá trị median (phân vị 50) (median).
- Giá trị upper quartile (phân vị 75) (upper_quartile).
- Giá trị max (max).

In [110]:
numeric_df=house_df.copy()
numeric_df=numeric_df[['GiaBan', 'DienTich', 'Gia/m2', 'PhongNgu', 'PhongVeSinh']]
columns=list(numeric_df.columns)
titles=['missing_ratio', 'min', 'lower_quartile', 'median', 'upper_quartile', 'max']
arrays=numeric_df.to_numpy()
num_col_dict={}
num_col_vals=[]
for i in range(len(columns)):
    temp=[]
    temp.append(((sum(np.isnan(arrays[:,i]))/arrays.shape[0])*100).round(3))
    temp.append(np.nanmin(arrays[:,i],axis=0).round(1))
    temp.append(np.nanpercentile(arrays[:,i],25,axis=0).round(3))
    temp.append(np.nanpercentile(arrays[:,i],50,axis=0).round(3))
    temp.append(np.nanpercentile(arrays[:,i],75,axis=0).round(3))
    temp.append(np.nanmax(arrays[:,i],axis=0).round(3))
    num_col_vals.append(temp)
for i in range(len(columns)):
    num_col_dict[columns[i]]=num_col_vals[i]
    num_col_dict['titles']=titles
numeric_info_df=pd.DataFrame(num_col_dict).set_index('titles')
numeric_info_df

,GiaBan,DienTich,Gia/m2,PhongNgu,PhongVeSinh
titles,,,,,
missing_ratio,0.00,0.0,0.000,0.0,16.652
min,0.80,1.0,12.900,1.0,1.000
lower_quartile,1.83,58.0,28.770,2.0,2.000
median,2.42,69.0,37.140,2.0,2.000
upper_quartile,3.45,81.0,49.785,2.0,2.000
max,130.00,385.0,3290.000,5.0,5.000


**Nhận xét:**
- Đa số các cột dạng số không có giá trị thiếu, do những giá trị thuộc tính này là những giá trị rất cần thiết và thông dụng khi quan tâm đến việc mua một căn hộ, chỉ có thuộc tính PhongVeSinh có phần trăm giá trị thiếu lên đến hơn 16%.
- Việc bỏ đi các trường hợp có thuộc tính nan trong trường hợp này là không khả thi do mất đi quá nhiều dữ liệu và phần tiền xử lý này sẽ được thực hiện ở bước học máy.

### 8. Xem xét sự phân bố giá trị của các cột dữ liệu không phải dạng số.

Việc thực hiện thống kê sẽ không bap gồm cột DiaChi do đã có cột Quan thay thế.

Thực hiện thống kê và lưu vào một dataframe với các dòng là đại diện cho các giá trị như sau:
- Tỉ lệ % (từ 0 đến 100) các giá trị thiếu (missing_ratio).
- Số lượng các giá trị khác nhau (không xét giá trị thiếu) (num_values).
- Tỉ lệ % (từ 0 đến 100) của mỗi giá trị được sort theo tỉ lệ % giảm dần (không xét giá trị thiếu, tỉ lệ là tỉ lệ so với số lượng các giá trị không thiếu): dùng dictionary để lưu, key là giá trị, value là tỉ lệ % (value_ratios).

In [25]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

categorical_df=house_df[['Quan','DuAn','TinhTrangBDS','TinhTrangGiayTo','TinhTrangNoiThat',
                         'HuongBanCong','HuongCua','DacDiem']]
col_names=list(categorical_df.columns)
titles=["missing_ratio", "num_values", "value_ratios"]
values_list=[]
dic={}
for i in col_names:
    df=categorical_df.copy()[[i]]
    size=len(df)
    new=list(df[i].values)
    df[i]=new
    temp=[]
    temp.append(((df[i].isnull().sum()/size)*100).round(3))
    df.dropna(inplace=True)
    vals=list(df[i].values)
    num_vals=list(set(vals))
    temp.append(len(num_vals))
    temp2={}
    new=[(df[i].value_counts()[j]/len(vals)*100).round(3) for j in num_vals]
    temp2={num_vals[j]:new[j] for j in range(len(num_vals))}
    temp2_sort={k:v for k,v in sorted(temp2.items(), key= lambda item:item[1], reverse=True)}
    temp.append(temp2_sort)
    values_list.append(temp)
dic={col_names[i]:values_list[i] for i in range(len(col_names))}
dic['titles']=titles
categorical_info_df=pd.DataFrame(dic).set_index('titles')
categorical_info_df

,Quan,DuAn,TinhTrangBDS,TinhTrangGiayTo,TinhTrangNoiThat,HuongBanCong,HuongCua,DacDiem
titles,,,,,,,,
missing_ratio,0.0,0.0,0.088,49.692,53.656,74.89,79.648,82.467
num_values,19,311,2,3,4,8,8,1
value_ratios,"{' Thành phố Thủ Đức': 23.26, ' Quận 7': 15.242, ' Quận 8': 11.982, ' Quận Tân Phú': 7.489, ' Quận Bình Thạnh': 7.313, ' Quận Bình Tân': 6.432, ' Quận 4': 4.053, ' Huyện Nhà Bè': 3.789, ' Quận 12': 3.524, ' Quận Tân Bình': 3.084, ' Huyện Bình Chánh': 2.907, ' Quận 6': 2.731, ' Quận 10': 2.115, ' Quận Gò Vấp': 1.85, ' Quận Phú Nhuận': 1.233, ' Quận 11': 1.057, ' Quận 5': 0.969, ' Quận 1': 0.529, ' Quận 3': 0.441}","{'Saigon Riverside Complex Q7': 3.877, 'Celadon City': 2.819, 'Vinhomes Grand Park': 2.379, 'Topaz Elite': 2.203, 'Sunrise City': 2.026, 'Akari City': 1.938, 'Saigon Gateway': 1.85, 'Vinhomes Central Park': 1.674, 'Richmond City': 1.498, 'The Sun Avenue': 1.41, 'Ricca': 1.322, 'Võ Đình Apartment': 1.322, 'Dream Home Riverside': 1.145, 'Topaz City': 1.057, 'Diamond Riverside (City Gate Towers 2)': 1.057, 'EHome 3': 1.057, 'The Gold View': 1.057, 'The Pegasuite': 1.057, 'Orchid Park': 1.057, 'Lavita Charm': 1.057, 'Eco Green Sài Gòn': 1.057, 'Mizuki Park': 0.969, 'LuxGarden': 0.969, 'Sài Gòn Intela': 0.793, 'Green Town Bình Tân': 0.793, 'River Gate': 0.793, 'Sunview Town': 0.793, 'The Pegasuite 2': 0.705, 'Hưng Ngân Garden': 0.705, 'The Everrich Infinity': 0.705, 'Căn hộ RichStar': 0.705, 'Hà Đô Centrosa Garden': 0.705, 'Dream Home Palace': 0.705, 'Wilton Tower': 0.617, 'The Botanica': 0.617, 'Căn hộ Cityland Parkhill Gò Vấp': 0.617, 'Botanica Premier': 0.617, 'City Garden': 0.617, 'Diamond Lotus Riverside': 0.617, 'Kingdom 101': 0.617, 'Flora Novia': 0.529, 'Q7 Boulevard': 0.529, 'Jamona City': 0.529, 'Khu căn hộ IDICO Tân Phú': 0.529, 'Empire City Thủ Thiêm': 0.529, 'De Capella': 0.529, 'The Park Residence': 0.529, '4S Riverside Linh Đông': 0.529, 'Osimi Tower': 0.529, 'The Origami – Vinhomes Grand Park': 0.529, 'Florita': 0.529, 'Carina Plaza': 0.441, 'Starlight Riverside': 0.441, 'The River Thủ Thiêm': 0.441, 'Hoàng Anh Gold House (Hoàng Anh An Tiến)': 0.441, 'M-One Nam Sài Gòn': 0.441, 'Central Premium': 0.441, 'Fresca Riverside': 0.441, 'Tara Residence': 0.441, 'The Eastern': 0.441, 'Sunrise Riverside': 0.441, 'An Gia Star': 0.441, 'Him Lam Chợ Lớn': 0.441, 'TaniBuilding Sơn Kỳ 1': 0.441, 'City Gate Towers': 0.441, 'Summer Square': 0.441, 'Chung cư Lê Thành': 0.441, 'Opal Riverside': 0.441, 'Masteri Thảo Điền': 0.441, 'Căn hộ DE LA SOL (Delasol)': 0.441, 'Đạt Gia Residence Thủ Đức': 0.441, 'La Astoria': 0.441, 'PiCity High Park': 0.352, 'Homyland 3 (Homyland Riverside)': 0.352, 'Asiana Capella': 0.352, 'Hưng Phát Silver Star': 0.352, 'The Golden Star': 0.352, 'Vinhomes Golden River BaSon': 0.352, 'The Western Capital': 0.352, 'Viva Riverside': 0.352, 'Jamila Khang Điền': 0.352, 'Conic Riverside': 0.352, 'Sadora Apartment': 0.352, 'Centum Wealth Complex': 0.352, 'Riva Park': 0.352, 'Lotus Sen Hồng ( Violet Tower )': 0.352, 'Docklands Sài Gòn': 0.352, 'Saigon Royal Residence': 0.352, 'One Verandah Mapletree': 0.352, 'Sunrise CityView': 0.352, 'Lavida Plus': 0.352, 'Phú Hoàng Anh': 0.352, 'Sky Garden 3': 0.352, 'The Tresor': 0.352, 'Central Plaza - 91 Phạm Văn Hai': 0.352, 'Topaz Garden': 0.352, 'Saigon Homes Bình Tân': 0.352, 'The Era Town (A1 Riverside)': 0.264, 'Căn hộ D’Lusso': 0.264, 'Safira Khang Điền (Sapphire Khang Điền)': 0.264, ...}","{'Đã bàn giao': 81.922, 'Chưa bàn giao': 18.078}","{'Đã có sổ': 64.799, 'Đang chờ sổ': 29.422, 'Giấy tờ khác': 5.779}","{'Nội thất đầy đủ': 39.924, 'Hoàn thiện cơ bản': 33.08, 'Nội thất cao cấp': 25.285, 'Bàn giao thô': 1.711}","{'Đông Nam': 27.018, 'Đông Bắc': 14.386, 'Đông': 12.632, 'Nam': 10.877, 'Bắc': 10.526, 'Tây Bắc': 9.825, 'Tây Nam': 9.123, 'Tây': 5.614}","{'Đông Nam': 17.316, 'Tây Bắc': 17.316, 'Đông Bắc': 14.286, 'Tây': 12.121, 'Nam': 11.255, 'Tây Nam': 10.823, 'Bắc': 10.39, 'Đông': 6.494}",{'Căn góc': 100.0}


## II. Một số trực quan cơ bản